In [4]:
# Load the cleaned tweet data
tweet_df = pd.read_csv("./data/AMZN_tweets.csv")  # Replace with your file path
tweet_df.head()

,Date,Tweet
0,2022-09-29 22:40:47+00:00,A group of lawmakers led by Sen. Elizabeth War...
1,2022-09-29 22:23:54+00:00,$NIO just because I'm down money doesn't mean ...
2,2022-09-29 18:34:51+00:00,Today’s drop in $SPX is a perfect example of w...
3,2022-09-29 15:57:59+00:00,Druckenmiller owned $CVNA this year \nMunger b...
4,2022-09-29 15:10:30+00:00,Top 10 $QQQ Holdings \n\nAnd Credit Rating\n\n...


In [5]:
def FinBERT_sentiment_score(text):
    """
    Compute sentiment score using pretrained FinBERT on -1 to 1 scale. 
    -1 being negative and 1 being positive.
    """
    tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')
    finbert = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')
    nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer, device=device)  # GPU support added

    # Perform sentiment analysis
    result = nlp(text)
    if result[0]["label"] == "positive":
        return result[0]["score"]
    elif result[0]["label"] == "negative":
        return -result[0]["score"]
    else:
        return 0



In [ ]:
# Add sentiment scores to the DataFrame
tweet_df['FinBERT score'] = tweet_sentiments

# Save the results to a CSV file
tweet_df.to_csv("tweet_sentiment_results.csv", index=False)

print("Sentiment analysis completed. Results saved to 'tweet_sentiment_results.csv'.")

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from tqdm import tqdm  # Import tqdm for progress tracking

# Check for GPU availability
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available

# Load the cleaned tweet data
tweet_df = pd.read_csv("./data/AMZN_tweets.csv")  # Replace with your file path
tweet_df.head()

def FinBERT_sentiment_score(text):
    """
    Compute sentiment score using pretrained FinBERT on -1 to 1 scale. 
    -1 being negative and 1 being positive.
    """
    tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')
    finbert = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')
    nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer, device=device)  # GPU support added

    # Perform sentiment analysis
    result = nlp(text)
    if result[0]["label"] == "positive":
        return result[0]["score"]
    elif result[0]["label"] == "negative":
        return -result[0]["score"]
    else:
        return 0

# Apply FinBERT sentiment analysis to tweets with progress tracking
tweet_sentiments = []
for tweet in tqdm(tweet_df['Tweet'], desc="Processing tweets", unit="tweet"):
    if isinstance(tweet, str) and tweet.strip():  # Ensure the tweet is a valid non-empty string
        score = FinBERT_sentiment_score(tweet)
        tweet_sentiments.append(score)
    else:
        tweet_sentiments.append(0)  # Default to neutral if tweet is empty or invalid

# Add sentiment scores to the DataFrame
tweet_df['FinBERT score'] = tweet_sentiments

# Save the results to a CSV file
tweet_df.to_csv("tweet_sentiment_results.csv", index=False)

print("Sentiment analysis completed. Results saved to 'tweet_sentiment_results.csv'.")


Processing tweets:  98%|█████████▊| 3992/4089 [1:14:00<01:50,  1.13s/tweet]  